In [1]:
import ipytest
from dataclasses import dataclass
from typing import TextIO, Generator

from collections import Counter
from collections.abc import Iterator

ipytest.autoconfig()

In [2]:
from pathlib import Path

In [3]:
data_path = Path.cwd() / 'data' / 'day5_input.txt'

In [4]:
data_path.exists()

True

In [5]:
@dataclass(frozen=True)
class PointCoordinates:
    x: int = 0
    y: int = 0

In [6]:
def row_iter(source: TextIO) -> Generator[list[str, str], None, None]:
    for line in source:
        yield line.strip('\n').split(' -> ')

In [7]:
def create_vent_lines(
    row_iter: Generator[list[str, str], None, None],
) -> Generator[tuple[PointCoordinates, PointCoordinates], None, None]:
    for starting_coordinate, ending_coordinate in row_iter:
        yield (
            PointCoordinates(
                x=int(starting_coordinate.split(',')[0]), y=int(starting_coordinate.split(',')[1])
            ),
            PointCoordinates(
                x=int(ending_coordinate.split(',')[0]), y=int(ending_coordinate.split(',')[1])
            ),
        )

In [8]:
def get_rows(file_path: Path) -> Iterator[tuple[PointCoordinates, PointCoordinates]]:
    with open(file_path, 'r') as file:
        yield from create_vent_lines(row_iter(file))

In [ ]:
%%ipytest -vv


def test_get_rows(tmp_path: Path) -> None:
    # Given
    file_text = """
        223,805 -> 223,548
        609,164 -> 609,503
        461,552 -> 796,552
        207,361 -> 207,34
    """
    test_file = tmp_path / 'test.txt'
    test_file.write_text(file_text.strip())

    # When
    result = list(get_rows(test_file))
    expected = [
        (PointCoordinates(x=223, y=805), PointCoordinates(x=223, y=548)),
        (PointCoordinates(x=609, y=164), PointCoordinates(x=609, y=503)),
        (PointCoordinates(x=461, y=552), PointCoordinates(x=796, y=552)),
        (PointCoordinates(x=207, y=361), PointCoordinates(x=207, y=34)),
    ]

    # Then
    assert len(result) == len(expected)
    assert all(
        actual[0].x == expected[0].x
        and actual[0].y == expected[0].y
        and actual[1].x == expected[1].x
        and actual[1].y == expected[1].y
        for (actual, expected) in zip(result, expected)
    )

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... 

collected 1 item

t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_get_rows PASSED                                  [100%]

======================================== 1 passed in 0.01s =========================================


In [9]:
vent_lines = list(get_rows(data_path))

In [ ]:
def generate_points_between(
    start: PointCoordinates, end: PointCoordinates, include_diagonals: bool = False
) -> list[None | PointCoordinates]:
    """
    Generate all points between two coordinates (start and end), inclusive.

    :param start: Starting point coordinates.
    :param end: Ending point coordinates.
    :param include_diagonals: If True, include diagonal points as well.
    :return: A list of PointCoordinates representing all points between start and end.
    """
    if not include_diagonals and start.x != end.x and start.y != end.y:
        return []  # Could raise an error, but the input does contain diagonal lines in Part 2.

    if include_diagonals and abs(start.x - end.x) != abs(start.y - end.y):
        raise ValueError('Diagonal lines must have equal x and y distance.')

    x1, y1 = start.x, start.y
    x2, y2 = end.x, end.y

    x_step = 1 if x2 > x1 else -1 if x2 < x1 else 0
    y_step = 1 if y2 > y1 else -1 if y2 < y1 else 0

    points = []
    x, y = x1, y1
    while (x, y) != (x2 + x_step, y2 + y_step):
        points.append(PointCoordinates(x=x, y=y))
        x += x_step
        y += y_step

    return points

In [21]:
%%ipytest -vv

import pytest


@pytest.mark.parametrize(
    'start, end, include_diagonals, expected',
    [
        (
            PointCoordinates(1, 1),
            PointCoordinates(1, 3),
            False,
            [PointCoordinates(1, 1), PointCoordinates(1, 2), PointCoordinates(1, 3)],
        ),
        (
            PointCoordinates(1, 1),
            PointCoordinates(3, 1),
            False,
            [PointCoordinates(1, 1), PointCoordinates(2, 1), PointCoordinates(3, 1)],
        ),
        (
            PointCoordinates(1, 1),
            PointCoordinates(3, 3),
            True,
            [PointCoordinates(1, 1), PointCoordinates(2, 2), PointCoordinates(3, 3)],
        ),
        (
            PointCoordinates(2, 2),
            PointCoordinates(1, 1),
            True,
            [PointCoordinates(2, 2), PointCoordinates(1, 1)],
        ),
    ],
)
def test_generate_points_between(start, end, include_diagonals, expected):
    # When
    result = generate_points_between(start, end, include_diagonals)

    # Then
    assert result == expected


def test_generate_points_between_for_invalid_inputs():
    # Given
    start = PointCoordinates(1, 1)
    end = PointCoordinates(3, 4)
    include_diagonals = True

    # When
    with pytest.raises(ValueError) as excinfo:
        generate_points_between(start, end, include_diagonals)

    # Then
    assert str(excinfo.value) == 'Diagonal lines must have equal x and y distance.'

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... collected 5 items

t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_generate_points_between[start0-end0-False-expected0] PASSED [ 20%]
t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_generate_points_between[start1-end1-False-expected1] PASSED [ 40%]
t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_generate_points_between[start2-end2-True-expected2] PASSED [ 60%]
t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_generate_points_between[start3-end3-True-expected3] PASSED [ 80%]
t_1b3cf327e5fa4730b8cd3d45223358b4.py::test_generate_points_between_for_invalid_inputs PASSED [100%]

======================================== 5 passed in 0

#### Part 1

In [11]:
point_counts = Counter(
    point
    for coordinate_pairs in vent_lines
    for point in generate_points_between(coordinate_pairs[0], coordinate_pairs[1])
)

In [12]:
overlapping_points = {point: count for point, count in point_counts.items() if count >= 2}

len(overlapping_points)

7142

#### Part 2

In [13]:
point_counts = Counter(
    point
    for coordinate_pairs in vent_lines
    for point in generate_points_between(
        coordinate_pairs[0], coordinate_pairs[1], include_diagonals=True
    )
)

In [14]:
overlapping_points = {point: count for point, count in point_counts.items() if count >= 2}

len(overlapping_points)

20012